In [1]:
# importing the libraries
import pandas as pd
import numpy as np

# for reading and displaying images
from skimage.io import imread
import matplotlib.pyplot as plt
%matplotlib inline

# for creating validation set
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupShuffleSplit

# for evaluating the model
from sklearn.metrics import accuracy_score
from tqdm import tqdm

from imutils import paths
from tqdm import tqdm

# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, BCELoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout, Sigmoid
from torch.optim import Adam, SGD

import os
import joblib
import torch
import random
import albumentations
import matplotlib.pyplot as plt
import argparse
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
from PIL import Image
from tqdm import tqdm
from torchvision import models as models
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary

In [5]:
!unzip /content/drive/MyDrive/Owen/rddc2020/yolov5/datasets/amsterdam/train/uncropped/train_set.zip

Streaming output truncated to the last 5000 lines.
  inflating: train_set/29624.jpeg    
  inflating: train_set/21214.jpeg    
  inflating: train_set/23229.jpeg    
  inflating: train_set/8893.jpeg     
  inflating: train_set/16167.jpeg    
  inflating: train_set/13365.jpeg    
  inflating: train_set/11358.jpeg    
  inflating: train_set/384.jpeg      
  inflating: train_set/24016.jpeg    
  inflating: train_set/2964.jpeg     
  inflating: train_set/32438.jpeg    
  inflating: train_set/3676.jpeg     
  inflating: train_set/30405.jpeg    
  inflating: train_set/29274.jpeg    
  inflating: train_set/6024.jpeg     
  inflating: train_set/32592.jpeg    
  inflating: train_set/4019.jpeg     
  inflating: train_set/35257.jpeg    
  inflating: train_set/16537.jpeg    
  inflating: train_set/20956.jpeg    
  inflating: train_set/21644.jpeg    
  inflating: train_set/23679.jpeg    
  inflating: train_set/26181.jpeg    
  inflating: train_set/24446.jpeg    
  inflating: train_set/8939.jpeg     

In [6]:
labelled_data = pd.read_csv("/content/train_set/train.csv")

In [7]:
labelled_data = labelled_data.drop("Unnamed: 0", axis="columns")
labelled_data = labelled_data.rename(columns={'Unnamed: 0.1':'image_name'})

In [8]:
labelled_data['image_path'] = "/content/train_set/" + labelled_data['image_name'].astype(str) + ".jpeg"
labelled_data['tensor_path'] = "/content/drive/MyDrive/Owen/prediction_maps/" + labelled_data['image_name'].astype(str) + ".pt"
labelled_data['final_tensor_path'] = "/content/drive/MyDrive/Owen/final_prediction_maps/" + labelled_data['image_name'].astype(str) + ".pt"

In [9]:
labelled_data["inspection_id"] = labelled_data['road_id'] +labelled_data['inspection_date'].astype("str")

In [10]:
gss = GroupShuffleSplit(n_splits=2, train_size=0.8/0.9, random_state=42)
gss.get_n_splits()
groups = np.array(labelled_data["road_id"])

In [11]:
train,test = next(gss.split(X = labelled_data, groups = groups))

In [12]:
xtrain = labelled_data.iloc[train].final_tensor_path.values
xtest = labelled_data.iloc[test].final_tensor_path.values
ytrain = labelled_data.iloc[train].cracking_binary.values
ytest = labelled_data.iloc[test].cracking_binary.values
train_inspections = labelled_data.iloc[train].inspection_id.values
test_inspections = labelled_data.iloc[test].inspection_id.values

In [13]:
class Net(Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer (output 320,000)
            Conv2d(1, 3, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(3),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=1),
            # Defining another 2D convolution layer (output 80,000)
            Conv2d(3, 4, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(4),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=1),
            # Defining another 2D convolution layer (output 20,000)
            Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(4),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=3),
            # Defining another 2D convolution layer (output 1,250)
            Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(4),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=3),
            # Defining another 2D convolution layer (output 1,250)
            Conv2d(4, 1, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(1),
            MaxPool2d(kernel_size=2, stride=2),
        )

        self.sigmoid = Sequential(
            Sigmoid()
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = self.sigmoid(x)
        x = torch.amax(x,dim=(1,2,3))
        return x

In [3]:
model = Net()
model.state_dict = torch.load("/content/drive/MyDrive/Owen/output/epoch39",map_location=torch.device('cpu'))
model.cnn_layers.requires_grad = False
if torch.cuda.is_available():
    model = model.cuda()

NameError: ignored

In [ ]:
#i = 0
#for tensor in tqdm(labelled_data['tensor_path'],total=len(labelled_data['tensor_path'])):
  #i = i+1
  #if i>21588:
    #newtensor = model.cnn_layers(torch.load(tensor)).squeeze(dim=0)
    #newpath = tensor[44:]
    #torch.save(newtensor, "/content/drive/MyDrive/Owen/final_prediction_maps/" + str(newpath))
  #else:
    #print("skip ", i)




  0%|          | 0/32710 [00:00<?, ?it/s]


  1%|▏         | 417/32710 [00:00<00:09, 3570.99it/s]

skip  1
skip  2
skip  3
skip  4
skip  5
skip  6
skip  7
skip  8
skip  9
skip  10
skip  11
skip  12
skip  13
skip  14
skip  15
skip  16
skip  17
skip  18
skip  19
skip  20
skip  21
skip  22
skip  23
skip  24
skip  25
skip  26
skip  27
skip  28
skip  29
skip  30
skip  31
skip  32
skip  33
skip  34
skip  35
skip  36
skip  37
skip  38
skip  39
skip  40
skip  41
skip  42
skip  43
skip  44
skip  45
skip  46
skip  47
skip  48
skip  49
skip  50
skip  51
skip  52
skip  53
skip  54
skip  55
skip  56
skip  57
skip  58
skip  59
skip  60
skip  61
skip  62
skip  63
skip  64
skip  65
skip  66
skip  67
skip  68
skip  69
skip  70
skip  71
skip  72
skip  73
skip  74
skip  75
skip  76
skip  77
skip  78
skip  79
skip  80
skip  81
skip  82
skip  83
skip  84
skip  85
skip  86
skip  87
skip  88
skip  89
skip  90
skip  91
skip  92
skip  93
skip  94
skip  95
skip  96
skip  97
skip  98
skip  99
skip  100
skip  101
skip  102
skip  103
skip  104
skip  105
skip  106
skip  107
skip  108
skip  109
skip  110
skip  11




  2%|▏         | 664/32710 [00:00<00:11, 2842.56it/s]


  3%|▎         | 911/32710 [00:00<00:12, 2523.43it/s]

664
skip  665
skip  666
skip  667
skip  668
skip  669
skip  670
skip  671
skip  672
skip  673
skip  674
skip  675
skip  676
skip  677
skip  678
skip  679
skip  680
skip  681
skip  682
skip  683
skip  684
skip  685
skip  686
skip  687
skip  688
skip  689
skip  690
skip  691
skip  692
skip  693
skip  694
skip  695
skip  696
skip  697
skip  698
skip  699
skip  700
skip  701
skip  702
skip  703
skip  704
skip  705
skip  706
skip  707
skip  708
skip  709
skip  710
skip  711
skip  712
skip  713
skip  714
skip  715
skip  716
skip  717
skip  718
skip  719
skip  720
skip  721
skip  722
skip  723
skip  724
skip  725
skip  726
skip  727
skip  728
skip  729
skip  730
skip  731
skip  732
skip  733
skip  734
skip  735
skip  736
skip  737
skip  738
skip  739
skip  740
skip  741
skip  742
skip  743
skip  744
skip  745
skip  746
skip  747
skip  748
skip  749
skip  750
skip  751
skip  752
skip  753
skip  754
skip  755
skip  756
skip  757
skip  758
skip  759
skip  760
skip  761
skip  762
skip  763
skip  




  4%|▎         | 1158/32710 [00:00<00:13, 2409.97it/s]


  4%|▍         | 1404/32710 [00:00<00:13, 2283.09it/s]

...




 63%|██████▎   | 20661/32710 [00:09<00:05, 2160.83it/s]


 64%|██████▍   | 20907/32710 [00:09<00:05, 2177.65it/s]

 20661
skip  20662
skip  20663
skip  20664
skip  20665
skip  20666
skip  20667
skip  20668
skip  20669
skip  20670
skip  20671
skip  20672
skip  20673
skip  20674
skip  20675
skip  20676
skip  20677
skip  20678
skip  20679
skip  20680
skip  20681
skip  20682
skip  20683
skip  20684
skip  20685
skip  20686
skip  20687
skip  20688
skip  20689
skip  20690
skip  20691
skip  20692
skip  20693
skip  20694
skip  20695
skip  20696
skip  20697
skip  20698
skip  20699
skip  20700
skip  20701
skip  20702
skip  20703
skip  20704
skip  20705
skip  20706
skip  20707
skip  20708
skip  20709
skip  20710
skip  20711
skip  20712
skip  20713
skip  20714
skip  20715
skip  20716
skip  20717
skip  20718
skip  20719
skip  20720
skip  20721
skip  20722
skip  20723
skip  20724
skip  20725
skip  20726
skip  20727
skip  20728
skip  20729
skip  20730
skip  20731
skip  20732
skip  20733
skip  20734
skip  20735
skip  20736
skip  20737
skip  20738
skip  20739
skip  20740
skip  20741
skip  20742
skip  20743
skip  207




 65%|██████▍   | 21154/32710 [00:09<00:05, 2047.69it/s]


 65%|██████▌   | 21401/32710 [00:09<00:05, 2105.00it/s]

skip  21155
skip  21156
skip  21157
skip  21158
skip  21159
skip  21160
skip  21161
skip  21162
skip  21163
skip  21164
skip  21165
skip  21166
skip  21167
skip  21168
skip  21169
skip  21170
skip  21171
skip  21172
skip  21173
skip  21174
skip  21175
skip  21176
skip  21177
skip  21178
skip  21179
skip  21180
skip  21181
skip  21182
skip  21183
skip  21184
skip  21185
skip  21186
skip  21187
skip  21188
skip  21189
skip  21190
skip  21191
skip  21192
skip  21193
skip  21194
skip  21195
skip  21196
skip  21197
skip  21198
skip  21199
skip  21200
skip  21201
skip  21202
skip  21203
skip  21204
skip  21205
skip  21206
skip  21207
skip  21208
skip  21209
skip  21210
skip  21211
skip  21212
skip  21213
skip  21214
skip  21215
skip  21216
skip  21217
skip  21218
skip  21219
skip  21220
skip  21221
skip  21222
skip  21223
skip  21224
skip  21225
skip  21226
skip  21227
skip  21228
skip  21229
skip  21230
skip  21231
skip  21232
skip  21233
skip  21234
skip  21235
skip  21236
skip  21237
skip

Streaming output truncated to the last 5000 lines.

 95%|█████████▍| 31044/32710 [1:08:47<08:19,  3.34it/s]


 95%|█████████▍| 31045/32710 [1:08:47<08:44,  3.18it/s]


 95%|█████████▍| 31046/32710 [1:08:47<08:06,  3.42it/s]


 95%|█████████▍| 31047/32710 [1:08:48<08:14,  3.36it/s]


 95%|█████████▍| 31048/32710 [1:08:48<07:56,  3.49it/s]


 95%|█████████▍| 31049/32710 [1:08:48<08:01,  3.45it/s]


 95%|█████████▍| 31050/32710 [1:08:48<08:00,  3.46it/s]


 95%|█████████▍| 31051/32710 [1:08:49<08:31,  3.24it/s]


 95%|█████████▍| 31052/32710 [1:08:49<08:21,  3.31it/s]


 95%|█████████▍| 31053/32710 [1:08:49<09:03,  3.05it/s]


 95%|█████████▍| 31054/32710 [1:08:50<09:13,  2.99it/s]


 95%|█████████▍| 31055/32710 [1:08:50<08:31,  3.23it/s]


 95%|█████████▍| 31056/32710 [1:08:50<08:03,  3.42it/s]


 95%|█████████▍| 31057/32710 [1:08:51<08:50,  3.12it/s]


 95%|█████████▍| 31058/32710 [1:08:51<09:03,  3.04it/s]


 95%|█████████▍| 31059/32710 [1:08:51<09:43,  2.83it/s]


 95%|█████████▍| 310

In [20]:
# image dataset module
class prediction_maps(Dataset):
    def __init__(self, path, labels, tfms=None):
        self.X = path
        self.y = labels

    def __len__(self):
        return (len(self.X))
    
    def __getitem__(self, i):
        tensor = torch.load(self.X[i])
        label = self.y[i]
        return tensor, torch.tensor(label, dtype=torch.long)

In [21]:
class final_Net(Module):   
    def __init__(self):
        super(final_Net, self).__init__()

        self.linear = Linear(in_features=21,out_features=1,bias=True)

        self.sigmoid = Sigmoid()

    # Defining the forward pass    
    def forward(self, x):
        x = torch.flatten(x, start_dim=1)
        x = self.linear(x)
        x = self.sigmoid(x)
        x = torch.max(x)
        return x

In [22]:
# defining the model
final_model = final_Net()
# defining the optimizer
optimizer = Adam(final_model.parameters(), lr=0.0003)
# defining the loss function
criterion = BCELoss()
# checking if GPU is available
if torch.cuda.is_available():
    final_model = final_model.cuda()
    criterion = criterion.cuda()
    
print(final_model)

final_Net(
  (linear): Linear(in_features=21, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [ ]:
from torchsummary import summary
summary(final_model, (1,3,7))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                    [-1, 1]              22
           Sigmoid-2                    [-1, 1]               0
Total params: 22
Trainable params: 22
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [23]:
''' SEED Everything '''
def seed_everything(SEED=42):
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.benchmark = True 
SEED=42
seed_everything(SEED=SEED)
''' SEED Everything '''
# set computation device
device = ('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Computation device: {device}")

Computation device: cuda:0


In [24]:
#validation function
def validate(model):
    print('Validating')
    model.eval()
    val_running_loss = 0.0
    val_running_correct = 0
    true_positives = 0
    true_negatives = 0
    false_positives = 0
    false_negatives = 0
    with torch.no_grad():
        for inspection in tqdm(np.unique(test_inspections),total=int(len(np.unique(test_inspections)))):
          test_data = prediction_maps(xtest[test_inspections==inspection], ytest[test_inspections==inspection], tfms=0)
          testloader = DataLoader(test_data,batch_size=len(test_data), shuffle=False)
          for i, data in enumerate(testloader):
            data, target = data[0].to(device), torch.max(data[1]).to(device)
            outputs = model(data)
            loss = criterion(outputs.float(), target.float())
            val_running_loss += loss.item()
            preds = torch.round(outputs.data)
            val_running_correct += (preds == target).sum().item()
            true_positives += ((preds == 1)&(target == 1)).sum().item()
            true_negatives += ((preds == 0)&(target == 0)).sum().item()
            false_positives += ((preds == 1)&(target == 0)).sum().item()
            false_negatives += ((preds == 0)&(target == 1)).sum().item()
        try:
          precision = true_positives/(true_positives+false_positives)
          recall = true_positives/(true_positives+false_negatives)
          f1 = 2 * (precision*recall)/(precision+recall)
        except:
          f1 = 0
          precision = 0
          recall = 0
        val_loss = val_running_loss/int(len(np.unique(test_inspections)))
        val_accuracy = 100. * val_running_correct/len(np.unique(test_inspections))
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}')
        print(f"Recall: {recall:.4f}, Precision: {precision:.2f}, F1: {f1:.2f}")
        
        return val_loss, val_accuracy, f1

In [25]:
# training function
def fit(model):
    print('Training')
    model.train()
    train_running_loss = 0.0
    train_running_correct = 0
    true_positives = 0
    true_negatives = 0
    false_positives = 0
    false_negatives = 0
    for inspection in tqdm(np.unique(train_inspections),total=int(len(np.unique(train_inspections)))):
        train_data = prediction_maps(xtrain[train_inspections==inspection], ytrain[train_inspections==inspection], tfms=1)
        trainloader = DataLoader(train_data, batch_size=len(train_data), shuffle=True)
        for i, data in enumerate(trainloader):
          data, target = data[0].to(device), torch.max(data[1]).to(device)
          optimizer.zero_grad()
          outputs = model(data)
          loss = criterion(outputs.float(), target.float())
          train_running_loss += loss.item()
          preds = torch.round(outputs.data)
          train_running_correct += (preds == target).sum().item()
          true_positives += ((preds == 1)&(target == 1)).sum().item()
          true_negatives += ((preds == 0)&(target == 0)).sum().item()
          false_positives += ((preds == 1)&(target == 0)).sum().item()
          false_negatives += ((preds == 0)&(target == 1)).sum().item()
          loss.backward()
          optimizer.step()
    try:
      precision = true_positives/(true_positives+false_positives)
      recall = true_positives/(true_positives+false_negatives)
      f1 = 2 * (precision*recall)/(precision+recall)
    except:
      precision = 0
      recall = 0
      f1 = 0
    train_loss = train_running_loss/int(len(np.unique(train_inspections)))
    train_accuracy = 100. * train_running_correct/len(np.unique(train_inspections))
    
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}")
    print(f"Recall: {recall:.4f}, Precision: {precision:.2f}, F1: {f1:.2f}")
    
    return train_loss, train_accuracy, f1

In [ ]:
train_loss , train_accuracy , train_f1 = [], [], []
val_loss , val_accuracy, val_f1 = [], [], []
start = time.time()
epochs = 75
for epoch in range(epochs):
    print(f"Epoch {epoch+1} of {epochs}")
    train_epoch_loss, train_epoch_accuracy, train_epoch_f1 = fit(final_model)
    val_epoch_loss, val_epoch_accuracy, val_epoch_f1 = validate(final_model)
    train_loss.append(train_epoch_loss)
    train_accuracy.append(train_epoch_accuracy)
    train_f1.append(train_epoch_f1)
    val_loss.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)
    val_f1.append(val_epoch_f1)
    path = "/content/drive/MyDrive/Owen/final_layer_output/epoch" + str(epoch)
    torch.save(final_model.state_dict(), path)
end = time.time()
print(f"{(end-start)/60:.3f} minutes")


  0%|          | 0/1690 [00:00<?, ?it/s]

Epoch 1 of 75
Training



 50%|█████     | 846/1690 [2:31:21<4:12:34, 17.96s/it]

In [ ]:
# accuracy plots
plt.figure(figsize=(10, 7))
plt.plot(train_accuracy, color='green', label='train accuracy')
plt.plot(val_accuracy, color='blue', label='validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
#plt.savefig('/content/drive/MyDrive/Owen/accuracy (2).png')
plt.show()
# loss plots
plt.figure(figsize=(10, 7))
plt.plot(train_loss, color='orange', label='train loss')
plt.plot(val_loss, color='red', label='validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
#plt.savefig('/content/drive/MyDrive/Owen/loss (2).png')
plt.show()
# F1 plots
plt.figure(figsize=(10, 7))
plt.plot(train_f1, color='orange', label='train F1')
plt.plot(val_f1, color='red', label='validation F1')
plt.xlabel('Epochs')
plt.ylabel('F1')
plt.legend()
#plt.savefig('/content/drive/MyDrive/Owen/F1 (2).png')
plt.show()

In [ ]:
columns = [str(i) for i in range(21)]
columns.append("inspection")
columns.append("target")

In [ ]:
inspection_level = pd.DataFrame(columns=columns)

for i,inspection in tqdm(enumerate(np.unique(train_inspections)),total=int(len(np.unique(train_inspections)))):
        inspection_level.loc[i,'inspection'] = inspection
        train_data = prediction_maps(xtrain[train_inspections==inspection], ytrain[train_inspections==inspection], tfms=1)
        trainloader = DataLoader(train_data, batch_size=len(train_data), shuffle=True)
        for data in trainloader:
          data, target = data[0].to(device).detach(), torch.max(data[1].to(device).detach())
          max = torch.max(data, dim = 0).values
          flat = torch.flatten(max)
          inspection_level.loc[i,0:21] = flat.tolist()
          inspection_level.loc[i,'target'] = target.item()

Streaming output truncated to the last 5000 lines.









 70%|███████   | 1191/1690 [2:32:18<49:22,  5.94s/it]









 71%|███████   | 1192/1690 [2:32:27<57:19,  6.91s/it]









 71%|███████   | 1193/1690 [2:32:30<46:57,  5.67s/it]









 71%|███████   | 1194/1690 [2:32:42<1:02:23,  7.55s/it]









 71%|███████   | 1195/1690 [2:32:54<1:13:16,  8.88s/it]









 71%|███████   | 1196/1690 [2:32:56<56:55,  6.91s/it]  









 71%|███████   | 1197/1690 [2:33:04<1:00:40,  7.38s/it]









 71%|███████   | 1198/1690 [2:33:07<49:37,  6.05s/it]  









 71%|███████   | 1199/1690 [2:33:16<57:05,  6.98s/it]









 71%|███████   | 1200/1690 [2:33:27<1:05:00,  7.96s/it]









 71%|███████   | 1201/1690 [2:33:36<1:07:41,  8.31s/it]









 71%|███████   | 1202/1690 [2:33:54<1:32:22, 11.36s/it]









 71%|███████   | 1203/1690 [2:33:59<1:15:36,  9.32s/it]









 71%|███████   | 1204/1690 [2:33:59<53:58,  6.66s/it]  









 71%|███████▏  | 1205/1690 [2:34:10<1:

In [ ]:
inspection_level.to_csv("/content/drive/MyDrive/Owen/road level max prediction maps.csv")